# Map of GMPH EOIs

Based on most recent EOI figures from Coursera sent 9th August 2018.

Total of 11691 EOIs.

Hover over the countries to see country totals and percentage breakdown.


In [8]:
import pandas as pd
import sys
import os
import plotly.offline as py
import plotly.graph_objs as go
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials

py.init_notebook_mode(connected=True)

scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('C:\\Users\\ikalkani\\Box\\Digital Learning Hub\\10. Data Analytics\\APIs\\gsheets\\ImperialDLH-504e4d642d14.json', scope)
gc = gspread.authorize(credentials)


In [9]:
def gsheet (key, sheet):
      
    book = gc.open_by_key(key)
    worksheet = book.worksheet(sheet)
    table = worksheet.get_all_values()
    
    df = pd.DataFrame(table, columns = table[0])
    df = df[1:]
    
    return df

In [13]:
EOI_countries = gsheet("1ncCQqj7Cb7ODFmwlnRQpD0PvUDeTZMITkUzQQfjR01I","Countries")
country_codes = gsheet("1yyufuG4M-iySTgboYSWCmN2jkP1BIB1b3XUQqjKX6i0","Simple")

EOI_countries = EOI_countries.merge(country_codes, left_on='Degree EoIs Eoi Response Country Cd', right_on='alpha-2', how='left')
EOI_countries['Degree EoIs Completed Eoi Responses'] = EOI_countries['Degree EoIs Completed Eoi Responses'].astype(float)

total = EOI_countries['Degree EoIs Completed Eoi Responses'].sum()

In [11]:
def countries (df, column, divide_by=total):

    data = [dict(
            type = 'choropleth',
            locations = df['alpha-3'],
            z = df[column],
            text = (df[column]/divide_by)*100,
            colorscale = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']],
            autocolorscale = False,
            marker = dict(
                line = dict (
                    color = 'rgb(180,180,180)',
                    width = 0.5
                ) ) ) ]

    layout = dict(
        height = 800,
        width = 800,
        geo = dict(
            showframe = True,
            showcoastlines = False,
            projection = dict(
                type = 'orthographic'
            )))

    fig = dict( data=data, layout=layout )
    plot = py.iplot(fig)

In [12]:
countries(EOI_countries, 'Degree EoIs Completed Eoi Responses')

______________________________________________________________
Contact i.kalkanis@imperial.ac.uk for more information.